In [1]:
import re
import requests
from lxml import etree
import math
import pandas as pd
import csv
import time

In [2]:
new_save_obj = {
    'city': [],
    'r_name': [],
    'username': [],
    'text': [],
    'address': [],
    'star': [],
    'src': [],
    'time': [],
    'friend_count': [],
    'review_count': [],
    'photo_count': []
}

In [3]:
with open('sampling_restaurants.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    
    for row in reader:
        print(row)
        city = row[0]
        r_name = row[1]
        link = re.findall('/biz/([\s\S]+)', row[5])[0]
        print(city, r_name, link)
        time.sleep(1)

        try:
            url = f'https://www.yelp.com/not_recommended_reviews/{link}?not_recommended_start=0'
            headers = {
                'Cookie': 'bse=aaf54a8504e5459ca9054b9b9f4710a5; hl=en_US; wdi=2|D2EB2C37FE7743B2|0x1.93bd4df9d0fdp+30|620d088025fbf980; g_state={"i_p":1694152357210,"i_l":3}; recentlocations=Napa%2C+CA%2C+US%3B%3BNapa%2C+CA%3B%3BScottsdale%2C+AZ%2C+US%3B%3Bscottsdale-az-us%3B%3BScottsdale%2C+AZ%3B%3BPhoenix%3B%3BPhoenix%2C%3B%3BPhoenix%2C+AZ; location=%7B%22parent_id%22%3A+45%2C+%22address2%22%3A+%22%22%2C+%22longitude%22%3A+-111.92734753081055%2C+%22unformatted%22%3A+%22Scottsdale%2CAz%2CUs%22%2C+%22country%22%3A+%22US%22%2C+%22state%22%3A+%22AZ%22%2C+%22min_longitude%22%3A+-111.96138300000001%2C+%22county%22%3A+%22Maricopa+County%22%2C+%22city%22%3A+%22Scottsdale%22%2C+%22zip%22%3A+%22%22%2C+%22max_longitude%22%3A+-111.78791197617181%2C+%22accuracy%22%3A+4%2C+%22address1%22%3A+%22%22%2C+%22provenance%22%3A+%22YELP_GEOCODING_ENGINE%22%2C+%22display%22%3A+%22Scottsdale%2C+AZ%22%2C+%22location_type%22%3A+%22locality%22%2C+%22max_latitude%22%3A+33.85662465458778%2C+%22latitude%22%3A+33.499404636990256%2C+%22min_latitude%22%3A+33.45888641584279%2C+%22place_id%22%3A+%221356%22%2C+%22address3%22%3A+%22%22%2C+%22borough%22%3A+%22%22%2C+%22isGoogleHood%22%3A+false%2C+%22language%22%3A+null%2C+%22neighborhood%22%3A+%22%22%2C+%22polygons%22%3A+null%2C+%22usingDefaultZip%22%3A+false%2C+%22confident%22%3A+null%7D; uuac=8Z-ed34nC9QzNfzXVSYaCk1Ms9Wriixl-3R4Z13bck8; xcj=1|scCpKe9qvNguwAq4c3UElPl46F8Qjv_-QrGY-qMpDyc; pid=5407bc2656dd1e95; bsi=1%7C89801f15-bb89-4c3d-a53c-b0d0d80ca82f%7C1693787054327%7C1693787054324; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Sep+04+2023+08%3A25%3A08+GMT%2B0800+(China+Standard+Time)&version=202304.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=05cf4f18-d283-4cd8-9fb8-4efb8c3dcc82&interactionCount=1&landingPath=https%3A%2F%2Fwww.yelp.com%2Fnot_recommended_reviews%2Fs-and-v-urban-italian-scottsdale%3Fnot_recommended_start%3D10&groups=BG82%3A1%2CC0003%3A1%2CC0002%3A1%2CC0001%3A1%2CC0004%3A1',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.62'
            }
            r = requests.get(url=url,headers=headers)
            tree = etree.HTML(r.text)
            try:
                count_info = tree.xpath('.//div[@class="ysection not-recommended-reviews review-list-wide"]/h3/text()')[0]
                count = re.findall('([0-9]*?) reviews', count_info)[0].strip()
                page = math.ceil(int(count) / 10)
                print(f'页数:{page}')
                al = tree.xpath('.//div[@class="ysection not-recommended-reviews review-list-wide"]/ul[@class="ylist ylist-bordered reviews"]/li')
                print(len(al))
            except:
                continue
            for a in al:
                try:
                    username = a.xpath('.//span[@class="user-display-name"]/text()')[0]
                except:
                    username = ''
                try:
                    text1 = a.xpath('.//div[@class="review-content"]/p/text()')
                    text2 = a.xpath(('.//div[@class="review-content"]/p/br/text()'))
                    text = ''.join(text1+text2)
                except:
                    text = ''
                try:
                    address = a.xpath('.//li[@class="user-location responsive-hidden-small"]/b/text()')[0]
                except:
                    address = ''
                try:
                    src = a.xpath('.//div[@class="photo-box pb-60s"]/img/@src')[0]
                except:
                    src = ''
                try:
                    star = a.xpath('.//div[@class="biz-rating__stars"]/div/@title')[0]
                    star = re.findall('([\s\S]*?)star rating', star)[0].strip()
                except:
                    star = ''
                try:
                    time1 = a.xpath('.//span[@class="rating-qualifier"]/text()')[0].strip()
                except:
                    time1 = ''
                try:
                    friend_count = a.xpath('.//li[@class="friend-count responsive-small-display-inline-block"]/b/text()')[0]
                except:
                    friend_count = ''
                try:
                    review_count = a.xpath('.//li[@class="review-count responsive-small-display-inline-block"]/b/text()')[0]
                except:
                    review_count = ''
                try:
                    photo_count = a.xpath('.//li[@class="photo-count responsive-small-display-inline-block"]/b/text()')[0]
                except:
                    photo_count = ''

                new_save_obj['city'].append(city)
                new_save_obj['r_name'].append(r_name)
                new_save_obj['username'].append(username)
                new_save_obj['text'].append(text)
                new_save_obj['address'].append(address)
                new_save_obj['src'].append(src)
                new_save_obj['star'].append(star)
                new_save_obj['time'].append(time1)
                new_save_obj['friend_count'].append(friend_count)
                new_save_obj['review_count'].append(review_count)
                new_save_obj['photo_count'].append(photo_count)
#                 try:
#                     print(username,address, text,star, time1, src, friend_count, review_count, photo_count)
#                 except:
#                     print('打不出来')
            df = pd.DataFrame(new_save_obj)
            df.to_csv('不推荐评论.csv', index=False)
        except:
            continue


        for i in range(1, int(page)):
            try:
                time.sleep(2)
                url1 = f'https://www.yelp.com/not_recommended_reviews/{link}?not_recommended_start={10*i}'
                print(url1)
                r = requests.get(url=url1,headers=headers)
                tree = etree.HTML(r.text)
                al = tree.xpath('.//div[@class="ysection not-recommended-reviews review-list-wide"]/ul[@class="ylist ylist-bordered reviews"]/li')
                print(len(al))
                for a in al:
                    try:
                        username = a.xpath('.//span[@class="user-display-name"]/text()')[0]
                    except:
                        username = ''
                    try:
                        text1 = a.xpath('.//div[@class="review-content"]/p/text()')
                        text2 = a.xpath(('.//div[@class="review-content"]/p/br/text()'))
                        text = ''.join(text1 + text2)
                    except:
                        text = ''
                    try:
                        address = a.xpath('.//li[@class="user-location responsive-hidden-small"]/b/text()')[0]
                    except:
                        address = ''
                    try:
                        src = a.xpath('.//div[@class="photo-box pb-60s"]/img/@src')[0]
                    except:
                        src = ''
                    try:
                        star = a.xpath('.//div[@class="biz-rating__stars"]/div/@title')[0]
                        star = re.findall('([\s\S]*?)star rating', star)[0].strip()
                    except:
                        star = ''
                    try:
                        time1 = a.xpath('.//span[@class="rating-qualifier"]/text()')[0].strip()
                    except:
                        time1 = ''
                    try:
                        friend_count = a.xpath('.//li[@class="friend-count responsive-small-display-inline-block"]/b/text()')[0]
                    except:
                        friend_count = ''
                    try:
                        review_count = a.xpath('.//li[@class="review-count responsive-small-display-inline-block"]/b/text()')[0]
                    except:
                        review_count = ''
                    try:
                        photo_count = a.xpath('.//li[@class="photo-count responsive-small-display-inline-block"]/b/text()')[0]
                    except:
                        photo_count = ''

                    new_save_obj['city'].append(city)
                    new_save_obj['r_name'].append(r_name)
                    new_save_obj['username'].append(username)
                    new_save_obj['text'].append(text)
                    new_save_obj['address'].append(address)
                    new_save_obj['src'].append(src)
                    new_save_obj['star'].append(star)
                    new_save_obj['time'].append(time1)
                    new_save_obj['friend_count'].append(friend_count)
                    new_save_obj['review_count'].append(review_count)
                    new_save_obj['photo_count'].append(photo_count)

#                     print(username, address, text, star, time1, src, friend_count, review_count, photo_count)
                df = pd.DataFrame(new_save_obj)
                df.to_csv('不推荐评论.csv', index=False)
            except:
                continue

['Alameda', 'Jong Ga House', '8.0', '4.0', "['Korean', 'Barbeque', 'Hot Pot']", '/biz/jong-ga-house-oakland', '2300.0']
Alameda Jong Ga House jong-ga-house-oakland
页数:13
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=10
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=20
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=30
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=40
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=50
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=60
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=70
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?not_recommended_start=80
10
https://www.yelp.com/not_recommended_reviews/jong-ga-house-oakland?

10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=700
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=710
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=720
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=730
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=740
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=750
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=760
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=770
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=780
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=790
10
https://www.yelp.com/not_recommended_reviews/homeroom-oakland?not_recommended_start=800

10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=90
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=100
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=110
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=120
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=130
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=140
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=150
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlington-arlington-2?not_recommended_start=160
10
https://www.yelp.com/not_recommended_reviews/busboys-and-poets-shirlin

10
https://www.yelp.com/not_recommended_reviews/the-best-sandwich-place-washington-5?not_recommended_start=90
10
https://www.yelp.com/not_recommended_reviews/the-best-sandwich-place-washington-5?not_recommended_start=100
10
https://www.yelp.com/not_recommended_reviews/the-best-sandwich-place-washington-5?not_recommended_start=110
3
['Alexandria', 'The Club Burgers & Sandwiches', '120.0', '4.2', "['Burgers', 'Sandwiches', 'Chicken Wings']", '/biz/the-club-burgers-and-sandwiches-alexandria', '268.0']
Alexandria The Club Burgers & Sandwiches the-club-burgers-and-sandwiches-alexandria
页数:2
10
https://www.yelp.com/not_recommended_reviews/the-club-burgers-and-sandwiches-alexandria?not_recommended_start=10
5
['Alexandria', 'Gordon Ramsay Fish & Chips', '119.0', '3.7', "['Fish & Chips', 'Comfort Food', 'Seafood']", '/biz/gordon-ramsay-fish-and-chips-washington', '275.0']
Alexandria Gordon Ramsay Fish & Chips gordon-ramsay-fish-and-chips-washington
页数:2
10
https://www.yelp.com/not_recommended_r

10
https://www.yelp.com/not_recommended_reviews/earls-kitchen-bar-boston-2?not_recommended_start=140
10
https://www.yelp.com/not_recommended_reviews/earls-kitchen-bar-boston-2?not_recommended_start=150
10
https://www.yelp.com/not_recommended_reviews/earls-kitchen-bar-boston-2?not_recommended_start=160
10
https://www.yelp.com/not_recommended_reviews/earls-kitchen-bar-boston-2?not_recommended_start=170
10
https://www.yelp.com/not_recommended_reviews/earls-kitchen-bar-boston-2?not_recommended_start=180
10
https://www.yelp.com/not_recommended_reviews/earls-kitchen-bar-boston-2?not_recommended_start=190
10
https://www.yelp.com/not_recommended_reviews/earls-kitchen-bar-boston-2?not_recommended_start=200
5
['Allston', 'Brothers & Sisters Co Coffeehouse + Cafe', '85.0', '4.0', "['Coffee & Tea', 'Bagels', 'Breakfast & Brunch']", '/biz/brothers-and-sisters-co-coffeehouse-cafe-brookline', '298.0']
Allston Brothers & Sisters Co Coffeehouse + Cafe brothers-and-sisters-co-coffeehouse-cafe-brookline


...

页数:4
10
https://www.yelp.com/not_recommended_reviews/smokey-mo-tucson?not_recommended_start=10
10
https://www.yelp.com/not_recommended_reviews/smokey-mo-tucson?not_recommended_start=20
10
https://www.yelp.com/not_recommended_reviews/smokey-mo-tucson?not_recommended_start=30
5
['Tucson', 'Agustin Kitchen', '36.0', '3.4', "['American (New)', 'French', 'Cocktail Bars']", '/biz/agustin-kitchen-tucson-3', '606.0']
Tucson Agustin Kitchen agustin-kitchen-tucson-3
页数:4
10
https://www.yelp.com/not_recommended_reviews/agustin-kitchen-tucson-3?not_recommended_start=10
10
https://www.yelp.com/not_recommended_reviews/agustin-kitchen-tucson-3?not_recommended_start=20
10
https://www.yelp.com/not_recommended_reviews/agustin-kitchen-tucson-3?not_recommended_start=30
6
['Tucson', '5 Points Market & Restaurant', '31.0', '4.3', "['American (New)', 'Breakfast & Brunch', 'Vegetarian']", '/biz/5-points-market-and-restaurant-tucson', '662.0']
Tucson 5 Points Market & Restaurant 5-points-market-and-restaurant-

10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=100
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=110
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=120
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=130
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=140
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=150
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=160
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recommended_start=170
10
https://www.yelp.com/not_recommended_reviews/jacks-restaurant-and-bar-pleasant-hill?not_recom

10
https://www.yelp.com/not_recommended_reviews/galpao-gaucho-brazilian-steakhouse-walnut-creek?not_recommended_start=40
10
https://www.yelp.com/not_recommended_reviews/galpao-gaucho-brazilian-steakhouse-walnut-creek?not_recommended_start=50
10
https://www.yelp.com/not_recommended_reviews/galpao-gaucho-brazilian-steakhouse-walnut-creek?not_recommended_start=60
1
['Walnut Creek', 'Stix Eatery', '111.0', '4.2', "['Salad', 'Noodles', 'Soup']", '/biz/stix-eatery-concord', '454.0']
Walnut Creek Stix Eatery stix-eatery-concord
页数:2
10
https://www.yelp.com/not_recommended_reviews/stix-eatery-concord?not_recommended_start=10
7
['Walnut Creek', 'Peony Garden', '127.0', '3.7', "['Dim Sum', 'Cantonese']", '/biz/peony-garden-walnut-creek', '386.0']
Walnut Creek Peony Garden peony-garden-walnut-creek
页数:3
10
https://www.yelp.com/not_recommended_reviews/peony-garden-walnut-creek?not_recommended_start=10
10
https://www.yelp.com/not_recommended_reviews/peony-garden-walnut-creek?not_recommended_start=2

10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=230
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=240
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=250
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=260
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=270
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=280
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=290
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=300
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=310
10
https://www.yelp.com/not_recommended_reviews/il-canale-washington-2?not_recommended_start=320
10
https://www.yelp.com/not_re

10
https://www.yelp.com/not_recommended_reviews/filomena-ristorante-washington?not_recommended_start=240
1
['Washington, DC', 'Lavagna', '73.0', '4.0', "['Italian']", '/biz/lavagna-washington', '755.0']
Washington, DC Lavagna lavagna-washington
页数:6
10
https://www.yelp.com/not_recommended_reviews/lavagna-washington?not_recommended_start=10
10
https://www.yelp.com/not_recommended_reviews/lavagna-washington?not_recommended_start=20
10
https://www.yelp.com/not_recommended_reviews/lavagna-washington?not_recommended_start=30
10
https://www.yelp.com/not_recommended_reviews/lavagna-washington?not_recommended_start=40
10
https://www.yelp.com/not_recommended_reviews/lavagna-washington?not_recommended_start=50
6
['Washington, DC', 'Lupo Verde', '57.0', '3.8', "['Italian', 'Salad', 'Wine Bars']", '/biz/lupo-verde-washington', '905.0']
Washington, DC Lupo Verde lupo-verde-washington
页数:11
10
https://www.yelp.com/not_recommended_reviews/lupo-verde-washington?not_recommended_start=10
10
https://www.

In [7]:
restaurant =pd.read_csv('sampling_restaurants.csv')

In [8]:
selected_rows = restaurant[restaurant['title'] == 'Bottega Louie']

# 打印筛选后的结果
print(selected_rows)

            city          title  rank  point  \
348  Los Angeles  Bottega Louie   1.0    4.1   

                                               tag  \
348  ['Italian', 'Desserts', 'Breakfast & Brunch']   

                               link  review_n  
348  /biz/bottega-louie-los-angeles   18400.0  


In [9]:
selected_rows.to_csv('BL.csv', index=False)

In [10]:
with open('BL.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    
    for row in reader:
        print(row)
        city = row[0]
        r_name = row[1]
        link = re.findall('/biz/([\s\S]+)', row[5])[0]
        print(city, r_name, link)
        time.sleep(1)

        try:
            url = f'https://www.yelp.com/not_recommended_reviews/{link}?not_recommended_start=0'
            headers = {
                'Cookie': 'bse=aaf54a8504e5459ca9054b9b9f4710a5; hl=en_US; wdi=2|D2EB2C37FE7743B2|0x1.93bd4df9d0fdp+30|620d088025fbf980; g_state={"i_p":1694152357210,"i_l":3}; recentlocations=Napa%2C+CA%2C+US%3B%3BNapa%2C+CA%3B%3BScottsdale%2C+AZ%2C+US%3B%3Bscottsdale-az-us%3B%3BScottsdale%2C+AZ%3B%3BPhoenix%3B%3BPhoenix%2C%3B%3BPhoenix%2C+AZ; location=%7B%22parent_id%22%3A+45%2C+%22address2%22%3A+%22%22%2C+%22longitude%22%3A+-111.92734753081055%2C+%22unformatted%22%3A+%22Scottsdale%2CAz%2CUs%22%2C+%22country%22%3A+%22US%22%2C+%22state%22%3A+%22AZ%22%2C+%22min_longitude%22%3A+-111.96138300000001%2C+%22county%22%3A+%22Maricopa+County%22%2C+%22city%22%3A+%22Scottsdale%22%2C+%22zip%22%3A+%22%22%2C+%22max_longitude%22%3A+-111.78791197617181%2C+%22accuracy%22%3A+4%2C+%22address1%22%3A+%22%22%2C+%22provenance%22%3A+%22YELP_GEOCODING_ENGINE%22%2C+%22display%22%3A+%22Scottsdale%2C+AZ%22%2C+%22location_type%22%3A+%22locality%22%2C+%22max_latitude%22%3A+33.85662465458778%2C+%22latitude%22%3A+33.499404636990256%2C+%22min_latitude%22%3A+33.45888641584279%2C+%22place_id%22%3A+%221356%22%2C+%22address3%22%3A+%22%22%2C+%22borough%22%3A+%22%22%2C+%22isGoogleHood%22%3A+false%2C+%22language%22%3A+null%2C+%22neighborhood%22%3A+%22%22%2C+%22polygons%22%3A+null%2C+%22usingDefaultZip%22%3A+false%2C+%22confident%22%3A+null%7D; uuac=8Z-ed34nC9QzNfzXVSYaCk1Ms9Wriixl-3R4Z13bck8; xcj=1|scCpKe9qvNguwAq4c3UElPl46F8Qjv_-QrGY-qMpDyc; pid=5407bc2656dd1e95; bsi=1%7C89801f15-bb89-4c3d-a53c-b0d0d80ca82f%7C1693787054327%7C1693787054324; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Sep+04+2023+08%3A25%3A08+GMT%2B0800+(China+Standard+Time)&version=202304.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=05cf4f18-d283-4cd8-9fb8-4efb8c3dcc82&interactionCount=1&landingPath=https%3A%2F%2Fwww.yelp.com%2Fnot_recommended_reviews%2Fs-and-v-urban-italian-scottsdale%3Fnot_recommended_start%3D10&groups=BG82%3A1%2CC0003%3A1%2CC0002%3A1%2CC0001%3A1%2CC0004%3A1',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.62'
            }
            r = requests.get(url=url,headers=headers)
            tree = etree.HTML(r.text)
            try:
                count_info = tree.xpath('.//div[@class="ysection not-recommended-reviews review-list-wide"]/h3/text()')[0]
                count = re.findall('([0-9]*?) reviews', count_info)[0].strip()
                page = math.ceil(int(count) / 10)
                print(f'页数:{page}')
                al = tree.xpath('.//div[@class="ysection not-recommended-reviews review-list-wide"]/ul[@class="ylist ylist-bordered reviews"]/li')
                print(len(al))
            except:
                continue
            for a in al:
                try:
                    username = a.xpath('.//span[@class="user-display-name"]/text()')[0]
                except:
                    username = ''
                try:
                    text1 = a.xpath('.//div[@class="review-content"]/p/text()')
                    text2 = a.xpath(('.//div[@class="review-content"]/p/br/text()'))
                    text = ''.join(text1+text2)
                except:
                    text = ''
                try:
                    address = a.xpath('.//li[@class="user-location responsive-hidden-small"]/b/text()')[0]
                except:
                    address = ''
                try:
                    src = a.xpath('.//div[@class="photo-box pb-60s"]/img/@src')[0]
                except:
                    src = ''
                try:
                    star = a.xpath('.//div[@class="biz-rating__stars"]/div/@title')[0]
                    star = re.findall('([\s\S]*?)star rating', star)[0].strip()
                except:
                    star = ''
                try:
                    time1 = a.xpath('.//span[@class="rating-qualifier"]/text()')[0].strip()
                except:
                    time1 = ''
                try:
                    friend_count = a.xpath('.//li[@class="friend-count responsive-small-display-inline-block"]/b/text()')[0]
                except:
                    friend_count = ''
                try:
                    review_count = a.xpath('.//li[@class="review-count responsive-small-display-inline-block"]/b/text()')[0]
                except:
                    review_count = ''
                try:
                    photo_count = a.xpath('.//li[@class="photo-count responsive-small-display-inline-block"]/b/text()')[0]
                except:
                    photo_count = ''

                new_save_obj['city'].append(city)
                new_save_obj['r_name'].append(r_name)
                new_save_obj['username'].append(username)
                new_save_obj['text'].append(text)
                new_save_obj['address'].append(address)
                new_save_obj['src'].append(src)
                new_save_obj['star'].append(star)
                new_save_obj['time'].append(time1)
                new_save_obj['friend_count'].append(friend_count)
                new_save_obj['review_count'].append(review_count)
                new_save_obj['photo_count'].append(photo_count)
#                 try:
#                     print(username,address, text,star, time1, src, friend_count, review_count, photo_count)
#                 except:
#                     print('打不出来')
            df = pd.DataFrame(new_save_obj)
            df.to_csv('BL_review.csv', index=False)
        except:
            continue


        for i in range(1, int(page)):
            try:
                time.sleep(2)
                url1 = f'https://www.yelp.com/not_recommended_reviews/{link}?not_recommended_start={10*i}'
                print(url1)
                r = requests.get(url=url1,headers=headers)
                tree = etree.HTML(r.text)
                al = tree.xpath('.//div[@class="ysection not-recommended-reviews review-list-wide"]/ul[@class="ylist ylist-bordered reviews"]/li')
                print(len(al))
                for a in al:
                    try:
                        username = a.xpath('.//span[@class="user-display-name"]/text()')[0]
                    except:
                        username = ''
                    try:
                        text1 = a.xpath('.//div[@class="review-content"]/p/text()')
                        text2 = a.xpath(('.//div[@class="review-content"]/p/br/text()'))
                        text = ''.join(text1 + text2)
                    except:
                        text = ''
                    try:
                        address = a.xpath('.//li[@class="user-location responsive-hidden-small"]/b/text()')[0]
                    except:
                        address = ''
                    try:
                        src = a.xpath('.//div[@class="photo-box pb-60s"]/img/@src')[0]
                    except:
                        src = ''
                    try:
                        star = a.xpath('.//div[@class="biz-rating__stars"]/div/@title')[0]
                        star = re.findall('([\s\S]*?)star rating', star)[0].strip()
                    except:
                        star = ''
                    try:
                        time1 = a.xpath('.//span[@class="rating-qualifier"]/text()')[0].strip()
                    except:
                        time1 = ''
                    try:
                        friend_count = a.xpath('.//li[@class="friend-count responsive-small-display-inline-block"]/b/text()')[0]
                    except:
                        friend_count = ''
                    try:
                        review_count = a.xpath('.//li[@class="review-count responsive-small-display-inline-block"]/b/text()')[0]
                    except:
                        review_count = ''
                    try:
                        photo_count = a.xpath('.//li[@class="photo-count responsive-small-display-inline-block"]/b/text()')[0]
                    except:
                        photo_count = ''

                    new_save_obj['city'].append(city)
                    new_save_obj['r_name'].append(r_name)
                    new_save_obj['username'].append(username)
                    new_save_obj['text'].append(text)
                    new_save_obj['address'].append(address)
                    new_save_obj['src'].append(src)
                    new_save_obj['star'].append(star)
                    new_save_obj['time'].append(time1)
                    new_save_obj['friend_count'].append(friend_count)
                    new_save_obj['review_count'].append(review_count)
                    new_save_obj['photo_count'].append(photo_count)

#                     print(username, address, text, star, time1, src, friend_count, review_count, photo_count)
                df = pd.DataFrame(new_save_obj)
                df.to_csv('BL_review.csv', index=False)
            except:
                continue

['Los Angeles', 'Bottega Louie', '1.0', '4.1', "['Italian', 'Desserts', 'Breakfast & Brunch']", '/biz/bottega-louie-los-angeles', '18400.0']
Los Angeles Bottega Louie bottega-louie-los-angeles
页数:439
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=10
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=20
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=30
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=40
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=50
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=60
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=70
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=80
10
https

0
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=820
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=830
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=840
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=850
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=860
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=870
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=880
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=890
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=900
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=910
1

10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1640
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1650
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1660
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1670
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1680
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1690
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1700
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1710
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=1720
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_s

10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2460
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2470
0
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2480
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2490
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2500
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2510
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2520
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2530
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=2540
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_st

10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3280
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3290
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3300
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3310
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3320
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3330
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3340
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3350
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=3360
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_s

10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4100
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4110
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4120
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4130
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4140
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4150
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4160
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4170
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_start=4180
10
https://www.yelp.com/not_recommended_reviews/bottega-louie-los-angeles?not_recommended_s